# Day 09. Exercise 00
# Regularization

## 0. Imports

In [290]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

## 1. Preprocessing

1. Read the file `dayofweek.csv` that you used in the previous day to a dataframe.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [291]:
df = pd.read_csv("../data/dayofweek.csv")
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,-0.788667,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.756764,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.724861,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.692958,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.661055,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,-0.533442,0.945382,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,-0.629151,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,-0.597248,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,-0.565345,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [292]:
x = df.drop("dayofweek", axis=1)
y = df["dayofweek"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

## 2. Logreg regularization

### a. Default regularization

1. Train a baseline model with the only parameters `random_state=21`, `fit_intercept=False`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model


The result of the code where you trained and evaluated the baseline model should be exactly like this (use `%%time` to get the info about how long it took to run the cell):

```
train -  0.62902   |   valid -  0.59259
train -  0.64633   |   valid -  0.62963
train -  0.63479   |   valid -  0.56296
train -  0.65622   |   valid -  0.61481
train -  0.63397   |   valid -  0.57778
train -  0.64056   |   valid -  0.59259
train -  0.64138   |   valid -  0.65926
train -  0.65952   |   valid -  0.56296
train -  0.64333   |   valid -  0.59701
train -  0.63674   |   valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
```

In [293]:
import numpy as np
def crossval(n_splits, x, y, model):
    kf = StratifiedKFold(n_splits=n_splits)
    accuracy_test_score = []

    for train_index, test_index in kf.split(x, y):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(x_train, y_train)
        predicts_train = model.predict(x_train)
        predicts_test = model.predict(x_test)
        accuracy_train = accuracy_score(y_train, predicts_train)
        accuracy_test = accuracy_score(y_test, predicts_test)
        print(f"train -  {accuracy_train:.5f}   |   test -  {accuracy_test:.5f}")
        accuracy_test_score.append(accuracy_test)

    print(f"Average accuracy on crossval is {np.mean(accuracy_test_score):.5f}")
    print(f"Std is {np.std(accuracy_test_score):.5f}")

In [294]:
model_lr = LogisticRegression(random_state=21, fit_intercept=False)

In [295]:
%%time
crossval(10, x_train, y_train, model_lr)

train -  0.62902   |   test -  0.59259
train -  0.64633   |   test -  0.62963
train -  0.63479   |   test -  0.56296
train -  0.65622   |   test -  0.61481
train -  0.63397   |   test -  0.57778
train -  0.64056   |   test -  0.59259
train -  0.64138   |   test -  0.65926
train -  0.65952   |   test -  0.56296
train -  0.64333   |   test -  0.59701
train -  0.63674   |   test -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
CPU times: user 7.05 s, sys: 194 ms, total: 7.24 s
Wall time: 839 ms


### b. Optimizing regularization parameters

1. In the cells below try different values of penalty: `none`, `l1`, `l2` – you can change the values of solver too.

In [296]:
model_lr = LogisticRegression(random_state=21, fit_intercept=False, penalty="none", solver="lbfgs", max_iter=1000)

In [297]:
%%time
crossval(10, x_train, y_train, model_lr)

train -  0.66612   |   test -  0.63704
train -  0.65787   |   test -  0.65926
train -  0.66694   |   test -  0.57778
train -  0.66529   |   test -  0.62963
train -  0.66694   |   test -  0.62222
train -  0.65952   |   test -  0.57778
train -  0.65045   |   test -  0.69630
train -  0.68425   |   test -  0.61481
train -  0.66474   |   test -  0.62687
train -  0.65651   |   test -  0.60448
Average accuracy on crossval is 0.62462
Std is 0.03379
CPU times: user 28.1 s, sys: 648 ms, total: 28.8 s
Wall time: 3.24 s


In [298]:
model_lr = LogisticRegression(random_state=21, fit_intercept=False, penalty="l1", solver="liblinear")

In [299]:
%%time
crossval(10, x_train, y_train, model_lr)

train -  0.61830   |   test -  0.54815
train -  0.62737   |   test -  0.62222
train -  0.60511   |   test -  0.54074
train -  0.63644   |   test -  0.62222
train -  0.62407   |   test -  0.55556
train -  0.62325   |   test -  0.58519
train -  0.61253   |   test -  0.63704
train -  0.64716   |   test -  0.58519
train -  0.63015   |   test -  0.59701
train -  0.61367   |   test -  0.59701
Average accuracy on crossval is 0.58903
Std is 0.03129
CPU times: user 2.38 s, sys: 49.1 ms, total: 2.43 s
Wall time: 273 ms


In [300]:
model_lr = LogisticRegression(random_state=21, fit_intercept=False, penalty="l2", solver="liblinear")

In [301]:
%%time
crossval(10, x_train, y_train, model_lr)

train -  0.61006   |   test -  0.56296
train -  0.61665   |   test -  0.61481
train -  0.61336   |   test -  0.59259
train -  0.62902   |   test -  0.60741
train -  0.60923   |   test -  0.55556
train -  0.61500   |   test -  0.57778
train -  0.61665   |   test -  0.61481
train -  0.64056   |   test -  0.53333
train -  0.62109   |   test -  0.58209
train -  0.61120   |   test -  0.57463
Average accuracy on crossval is 0.58160
Std is 0.02532
CPU times: user 1.65 s, sys: 44.4 ms, total: 1.7 s
Wall time: 190 ms


## 3. SVM regularization

### a. Default regularization

1. Train a baseline model with the only parameters `probability=True`, `kernel='linear'`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [302]:
model_svc = SVC(random_state=21, probability=True, kernel="linear")

In [303]:
%%time
crossval(10, x_train, y_train, model_svc)

train -  0.70486   |   test -  0.65926
train -  0.69662   |   test -  0.75556
train -  0.69415   |   test -  0.62222
train -  0.70239   |   test -  0.65185
train -  0.69085   |   test -  0.65185
train -  0.68920   |   test -  0.64444
train -  0.69250   |   test -  0.72593
train -  0.70074   |   test -  0.62222
train -  0.69605   |   test -  0.61940
train -  0.71087   |   test -  0.63433
Average accuracy on crossval is 0.65871
Std is 0.04359
CPU times: user 6.91 s, sys: 24.4 ms, total: 6.94 s
Wall time: 9.2 s


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `C`.

In [304]:
model_svc = SVC(random_state=21, probability=True, kernel="linear", C=10.0)

In [305]:
%%time
crossval(10, x_train, y_train, model_svc)

train -  0.75021   |   test -  0.72593
train -  0.77741   |   test -  0.82963
train -  0.78566   |   test -  0.68148
train -  0.76834   |   test -  0.73333
train -  0.75185   |   test -  0.77778
train -  0.75598   |   test -  0.68889
train -  0.76257   |   test -  0.74074
train -  0.77411   |   test -  0.68889
train -  0.78254   |   test -  0.71642
train -  0.78418   |   test -  0.69403
Average accuracy on crossval is 0.72771
Std is 0.04417
CPU times: user 8.72 s, sys: 10.4 ms, total: 8.74 s
Wall time: 8.67 s


In [306]:
model_svc = SVC(random_state=21, probability=True, kernel="linear", C=0.1)

In [307]:
%%time
crossval(10, x_train, y_train, model_svc)

train -  0.58120   |   test -  0.55556
train -  0.57543   |   test -  0.56296
train -  0.57378   |   test -  0.57037
train -  0.59275   |   test -  0.57037
train -  0.58120   |   test -  0.54815
train -  0.57955   |   test -  0.54815
train -  0.57296   |   test -  0.61481
train -  0.59192   |   test -  0.54815
train -  0.59967   |   test -  0.52985
train -  0.57825   |   test -  0.57463
Average accuracy on crossval is 0.56230
Std is 0.02177
CPU times: user 5.54 s, sys: 16.7 ms, total: 5.55 s
Wall time: 5.17 s


In [308]:
model_svc = SVC(random_state=21, probability=True, kernel="linear", C=0.5)

In [309]:
%%time
crossval(10, x_train, y_train, model_svc)

train -  0.66694   |   test -  0.63704
train -  0.66612   |   test -  0.73333
train -  0.67271   |   test -  0.60741
train -  0.67354   |   test -  0.62963
train -  0.67766   |   test -  0.64444
train -  0.66529   |   test -  0.61481
train -  0.66200   |   test -  0.68889
train -  0.66529   |   test -  0.57037
train -  0.67463   |   test -  0.59701
train -  0.66804   |   test -  0.61194
Average accuracy on crossval is 0.63349
Std is 0.04471
CPU times: user 5.38 s, sys: 12.6 ms, total: 5.39 s
Wall time: 5.02 s


## 4. Tree

### a. Default regularization

1. Train a baseline model with the only parameter `max_depth=10` and `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [310]:
model_tree = DecisionTreeClassifier(max_depth=10, random_state=21)

In [311]:
%%time
crossval(10, x_train, y_train, model_tree)

train -  0.81039   |   test -  0.74074
train -  0.77741   |   test -  0.74074
train -  0.83347   |   test -  0.70370
train -  0.79720   |   test -  0.76296
train -  0.82440   |   test -  0.75556
train -  0.80379   |   test -  0.68889
train -  0.80709   |   test -  0.76296
train -  0.80132   |   test -  0.65926
train -  0.80807   |   test -  0.75373
train -  0.80478   |   test -  0.68657
Average accuracy on crossval is 0.72551
Std is 0.03562
CPU times: user 153 ms, sys: 12.9 ms, total: 166 ms
Wall time: 152 ms


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `max_depth`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [312]:
model_tree = DecisionTreeClassifier(max_depth=1, random_state=21)

In [313]:
%%time
crossval(10, x_train, y_train, model_tree)

train -  0.35367   |   test -  0.37037
train -  0.35449   |   test -  0.36296
train -  0.35614   |   test -  0.34815
train -  0.35449   |   test -  0.36296
train -  0.35532   |   test -  0.35556
train -  0.35367   |   test -  0.37037
train -  0.35532   |   test -  0.35556
train -  0.35614   |   test -  0.34815
train -  0.35667   |   test -  0.34328
train -  0.35750   |   test -  0.33582
Average accuracy on crossval is 0.35532
Std is 0.01094
CPU times: user 122 ms, sys: 12.9 ms, total: 135 ms
Wall time: 123 ms


In [314]:
model_tree = DecisionTreeClassifier(max_depth=50, random_state=21)

In [315]:
%%time
crossval(10, x_train, y_train, model_tree)

train -  1.00000   |   test -  0.85926
train -  1.00000   |   test -  0.91852
train -  1.00000   |   test -  0.86667
train -  1.00000   |   test -  0.91111
train -  1.00000   |   test -  0.88148
train -  1.00000   |   test -  0.85185
train -  1.00000   |   test -  0.92593
train -  1.00000   |   test -  0.88148
train -  1.00000   |   test -  0.88060
train -  1.00000   |   test -  0.88060
Average accuracy on crossval is 0.88575
Std is 0.02374
CPU times: user 170 ms, sys: 4.35 ms, total: 175 ms
Wall time: 157 ms


In [316]:
model_tree = DecisionTreeClassifier(max_depth=21, random_state=21)

In [317]:
%%time
crossval(10, x_train, y_train, model_tree)

train -  0.99176   |   test -  0.86667
train -  0.99176   |   test -  0.90370
train -  0.99093   |   test -  0.85926
train -  0.99176   |   test -  0.89630
train -  0.99093   |   test -  0.88148
train -  0.98516   |   test -  0.88148
train -  0.98928   |   test -  0.91852
train -  0.99423   |   test -  0.87407
train -  0.99588   |   test -  0.86567
train -  0.99012   |   test -  0.88060
Average accuracy on crossval is 0.88278
Std is 0.01757
CPU times: user 175 ms, sys: 4.51 ms, total: 179 ms
Wall time: 158 ms


## 5. Random forest

### a. Default regularization

1. Train a baseline model with the only parameters `n_estimators=50`, `max_depth=14`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [318]:
model_forest = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)   

In [319]:
%%time
crossval(10, x_train, y_train, model_forest)

train -  0.96455   |   test -  0.88148
train -  0.96208   |   test -  0.91852
train -  0.96785   |   test -  0.86667
train -  0.96455   |   test -  0.89630
train -  0.96538   |   test -  0.91111
train -  0.96538   |   test -  0.88148
train -  0.97115   |   test -  0.91852
train -  0.96867   |   test -  0.85185
train -  0.97364   |   test -  0.88060
train -  0.97941   |   test -  0.86567
Average accuracy on crossval is 0.88722
Std is 0.02204
CPU times: user 1.81 s, sys: 9.69 ms, total: 1.82 s
Wall time: 1.63 s


### b. Optimizing regularization parameters

1. In the new cells try different values of the parameters `max_depth` and `n_estimators`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [320]:
model_forest = RandomForestClassifier(n_estimators=14, max_depth=50, random_state=21)   

In [321]:
%%time
crossval(10, x_train, y_train, model_forest)

train -  1.00000   |   test -  0.89630
train -  0.99918   |   test -  0.94074
train -  1.00000   |   test -  0.91111
train -  0.99753   |   test -  0.93333
train -  1.00000   |   test -  0.90370
train -  1.00000   |   test -  0.87407
train -  0.99918   |   test -  0.91852
train -  1.00000   |   test -  0.89630
train -  1.00000   |   test -  0.91791
train -  0.99835   |   test -  0.89552
Average accuracy on crossval is 0.90875
Std is 0.01878
CPU times: user 656 ms, sys: 4.68 ms, total: 661 ms
Wall time: 591 ms


In [322]:
model_forest = RandomForestClassifier(n_estimators=21, max_depth=21, random_state=21)   

In [323]:
%%time
crossval(10, x_train, y_train, model_forest)

train -  0.99670   |   test -  0.88148
train -  0.99753   |   test -  0.95556
train -  0.99505   |   test -  0.89630
train -  0.99670   |   test -  0.92593
train -  0.99505   |   test -  0.89630
train -  0.99588   |   test -  0.88148
train -  0.99588   |   test -  0.91852
train -  0.99588   |   test -  0.89630
train -  0.99671   |   test -  0.91791
train -  0.99506   |   test -  0.88806
Average accuracy on crossval is 0.90578
Std is 0.02227
CPU times: user 890 ms, sys: 9.08 ms, total: 899 ms
Wall time: 803 ms


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.
3. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your test dataset).
4. Save the model.

In [324]:
model_forest = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)   
model_forest.fit(x_train, y_train)
predict = model_forest.predict(x_test)
accuracy_score(predict, y_test)

0.908284023668639

In [325]:
df_analyze = pd.DataFrame()
df_analyze["dayofweek"] = y_test
df_analyze["dayofweek_predict"] = predict
df_analyze
errors = (df_analyze[df_analyze["dayofweek"] != df_analyze["dayofweek_predict"]]["dayofweek"].value_counts() / len(predict)) * 100
errors

6    2.071006
1    2.071006
0    2.071006
4    1.183432
5    0.887574
2    0.591716
3    0.295858
Name: dayofweek, dtype: float64

In [326]:
joblib.dump(model_forest, "model.joblib")

['model.joblib']